In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

from sklearn import metrics
import math
import numpy as np
from tensorflow.keras.models import Model

from tensorflow.keras.applications.nasnet import NASNetLarge, preprocess_input

In [2]:
pixels = 331

# Input pixel dimensions.  All training and test examples will be resized to (pixel, pixel, 3)
conv_base = NASNetLarge(weights='imagenet', include_top=False, input_shape=(pixels,pixels,3))

conv_base.trainable = False


In [3]:
sf40_dir = "./Sanford40"
BATCH_SIZE = 20

sf40_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range= [0.9,1.1],
    horizontal_flip=True,
    validation_split=0.3) # set validation split

train_generator = sf40_datagen.flow_from_directory(
    sf40_dir,
    batch_size=BATCH_SIZE, 
    target_size=(pixels, pixels),
    class_mode = 'categorical',
    subset='training') # set as training data

validation_generator = sf40_datagen.flow_from_directory(
    sf40_dir, 
    batch_size=BATCH_SIZE, 
    target_size=(pixels, pixels),
    class_mode = 'categorical',
    subset='validation') # set as validation data

y_true = validation_generator.classes

train_m = len(train_generator.classes)
valid_m = len(validation_generator.classes)

mapping = dict()
for activity, idx in train_generator.class_indices.items():
    mapping[idx] = activity

train_steps = math.ceil(train_m/BATCH_SIZE)
valid_steps = math.ceil(valid_m/BATCH_SIZE)

Found 6693 images belonging to 40 classes.
Found 2839 images belonging to 40 classes.


In [4]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Conv2D(1024, (3, 3), padding="same", strides=(1, 1), activation="relu", name="ClassConv"))
model.add(layers.GlobalAveragePooling2D(name="GAP"))
model.add(layers.Dense(40, activation="softmax", name="class"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Functional)          (None, 11, 11, 4032)      84916818  
_________________________________________________________________
ClassConv (Conv2D)           (None, 11, 11, 1024)      37159936  
_________________________________________________________________
GAP (GlobalAveragePooling2D) (None, 1024)              0         
_________________________________________________________________
class (Dense)                (None, 40)                41000     
Total params: 122,117,754
Trainable params: 37,200,936
Non-trainable params: 84,916,818
_________________________________________________________________


In [5]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


filepath = "models/class_only/checkpoints/epoch_{epoch:02d}-{val_loss:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min')

#callback = tf.keras.callbacks.EarlyStopping(monitor='classification_loss', patience=5)

logdir = "models/class_only/logs"

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4), metrics = ['acc'])

In [ ]:
epochs = 20
history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=epochs, validation_data=validation_generator, validation_steps=valid_steps, callbacks=[tensorboard_callback, checkpoint])

Epoch 1/20
335/335 [==============================] - 2571s 8s/step - loss: 0.8977 - acc: 0.7478 - val_loss: 0.7019 - val_acc: 0.7996

Epoch 00001: val_loss improved from inf to 0.70186, saving model to models/class_only/checkpoints\epoch_01-0.70.h5
Epoch 2/20
335/335 [==============================] - 2557s 8s/step - loss: 0.4201 - acc: 0.8714 - val_loss: 0.6203 - val_acc: 0.8232

Epoch 00002: val_loss improved from 0.70186 to 0.62030, saving model to models/class_only/checkpoints\epoch_02-0.62.h5
Epoch 3/20
335/335 [==============================] - 2562s 8s/step - loss: 0.2579 - acc: 0.9163 - val_loss: 0.6358 - val_acc: 0.8193

Epoch 00003: val_loss did not improve from 0.62030
Epoch 4/20
335/335 [==============================] - 2554s 8s/step - loss: 0.1771 - acc: 0.9411 - val_loss: 0.6895 - val_acc: 0.8218

Epoch 00004: val_loss did not improve from 0.62030
Epoch 5/20
335/335 [==============================] - 2521s 8s/step - loss: 0.1262 - acc: 0.9588 - val_loss: 0.7028 - val_ac

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
 BATCH_SIZE = 12

pixels = 224
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=10, zoom_range= [0.9,1.1])
test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = multiple_outputs(test_datagen, 
                                   image_dir=sf40_dir, 
                                   batch_size=BATCH_SIZE, 
                                   image_size=pixels)

valid_temp = test_datagen.flow_from_directory(sf40_dir, batch_size=BATCH_SIZE, target_size=(pixels,pixels), class_mode = 'categorical', shuffle=False)
y_true = valid_temp.classes

valid_m = len(valid_temp.classes)

mapping = dict()
for activity, idx in valid_temp.class_indices.items():
    mapping[idx] = activity


valid_steps = math.ceil(valid_m/BATCH_SIZE)

In [ ]:
BATCH_SIZE = 12

valid_generator = test_datagen.flow_from_directory(sf40_dir, batch_size=BATCH_SIZE, target_size=(pixels,pixels), class_mode = 'categorical', shuffle=False)

y_true = valid_generator.classes

valid_m = len(valid_generator.classes)

mapping = dict()
for activity, idx in train_generator.class_indices.items():
    mapping[idx] = activity

valid_steps = math.ceil(valid_m/BATCH_SIZE)

In [ ]:
predictions = model.predict_generator(valid_generator, valid_steps)
predictions = predictions.argmax(axis=1)
matrix = metrics.confusion_matrix(y_true, predictions)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(matrix, index = [mapping[i] for i in range(40)], columns = [mapping[i] for i in range(40)])
plt.figure(figsize = (40, 40))
sn.heatmap(df_cm, annot=True)